<div id="colab_button">
  <h1>Uploading models</h1>
  <a target="_blank" href="LINK_GOOGLE_COLAB"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>

_____________________________________________________________________

You've tested and installed BlindAI? Time to upload your model on your server instance! 

You can do it in two lines of code using our Python API, but first your model will need to be converted to the Open Neural Network Exchange Format (ONNX) format. This is because ONNX is a standard enabling framework interoperability, allowing you to easily move models between different machine learning libraries.

In this tutorial, we will show you how to take models from three of the most popular ML libraries, `PyTorch`, `TensorFlow` and `HuggingFace`, convert them to ONNX format, and upload them to BlindAI.

Let's dive in!

## Pre Requisites
_______________________________________

### Installing required dependencies

Unless you're are running this notebook on [Google Colab](LINK_GOOGLE_COLAB), you'll need to have [`Python`](https://www.python.org/downloads/) (3.8 or greater) and [`pip`](https://pypi.org/project/pip/) installed to run this notebook.

Then, you'll need to install the BlindAI-preview package.

In [ ]:
# install blindai-preview package
!pip install blindai-preview

We will also need to install some additional dependencies for this notebook:

- [`torch`](https://pytorch.org/): to demonstrate ONNX conversion for PyTorch models
- [`tensorflow`](https://www.tensorflow.org/), `tensorflow_hub`(https://tfhub.dev/) and `tf2onnx`(https://github.com/onnx/tensorflow-onnx): to demonstrate ONNX conversion for TensorFlow models
- [`optimum[exporters]`](https://huggingface.co/): to demonstrate ONNX conversion for HuggingFace models

In [ ]:
# install all other required packages
!pip install torch tensorflow tf2onnx optimum[exporters]

### Launch the BlindAI server

Let's launch an instance of BlindAI's server so we can upload the model. 

For the purposes of this tutorial, we will be using the `blindai_preview.testing` server which has been designed for testing purposes *only*.

In [ ]:
# import testing submodule
import blindai_preview.testing

# start the server
srv = blindai_preview.testing.start_mock_server()

BlindAI mock server (version 0.0.8) already installed


>Note that the blindai-preview testing module launches the server in simulation mode. As a result, it doesn't provide hardware protection and **must not be used in production**. We created this option to enable users to quickly and easily test BlindAI without needing to run the server on a machine with Intel SGX hardware. To learn more about how to run the server with hardware protections on, see [our documentation](https://blindai-preview.mithrilsecurity.io/en/latest/).

## PyTorch model
__________________

We will use the pretrained `resnet18` neural network as our example model for this section. The `resnet18` model classifies images and returns possbible labels for the image with their probability of being the correct label.

### Download the model

We'll download the model from `PyTorch Hub`, by using the `hub` module's `load()` method. 

The first argument we provide specifies the GitHub repo and directory where the model can be installed from. The second specifies the name of the model to be downloaded. Then we set the `pretrained` option to `True`.

In [ ]:
import torch

model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

Before exporting, it's very important to ensure the model is set to inference mode. We do that by calling `model.eval()` or `model.train(False)`.

In [ ]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### ONNX conversion

To convert our model to ONNX fomat, we will use the `torch.onnx.export()` function.

The `export()` method will execute the model and record a trace of what operators are used to compute the outputs. In order for `export` to be able to perform this "dry-run", we need to supply it with some dummy inputs in the shape the model would expect.

For `resnet18`, the dummy input should be in the following format: `batch_size`, `channel_width`, `image_size`, `image_size`.

The values we provide for each dimension, in this case `batch_size`, `channel_width`, `image_size` and `image_size` will be fixed in the exported ONNX graph, meaning all future input to the model must match this shape. For example, if we set the `batch_size` to 1, users will only be able to upload one image at a time to be inferenced by the model.

There is a way to avoid having to set a fixed value though, by specifying that a certain axes is a dynamic axes in the `export` options. For example, if we specify `batch_size` as a dynamic axes, users will be able to upload one or multiple images to the model at a time.

Let's create our dummy input now:

In [ ]:
# create dummy inputs for resnet18 model
dummy_inputs = torch.zeros(1,3,224,224)

The channel width should be `3, representing the RGB values of our image, and the image size should be `224x224`. 

We will set the `batch_size` to `1`, but then specify the first dimension as dynamic with the `dynamic_axes` option.

Since the values used for our dummy input are not important in this tutorial, we will set fill them with `0`.

Now that we have created our dummy inputs, we are ready to call the `onnx.export()` method. We pass the method:
- our PyTorch model,
- the dummy inputs,
- the name we want to give our ONNX file,
- any dynamic axes for input and output values.

In [ ]:
torch.onnx.export(model, dummy_inputs, 
                "resnet18.onnx", 
                dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

> For information about more export options, see the `torch.onnx.export` [documentation](https://pytorch.org/docs/stable/onnx.html#torch.onnx.export).

Our PyTorch model has been succesfully converted!

### Upload the model

To upload the model, we need to connect to the server using BlindAI's `core.connect()` function.

Setting up a production server is not the focus of this tutorial, which is why we've been using a server in `simulation` mode. This means two things: 

- We also need to set the `simulation_mode` parameter to `True` on the client side. This is needed because the client will refuse to connect to an unsecure server otherwise.  
- We also set the `hazmat_http_on_unattested_port` option to `True`. 

  >By default, the `blindai_preview` package requires a HTTPS connection for communications between the client and server on the unattested port 9923. But for testing purposes we opt out of this requirement and connect without a secure connection. This should **not be done in production**, please refer to our documentation to set up a production server.

In [ ]:
import blindai_preview 
import blindai_preview

# AI company connects
client_1 = blindai_preview.core.connect(addr="localhost", simulation_mode=True, hazmat_http_on_unattested_port=True)

/usr/local/lib/python3.9/dist-packages/blindai_preview/client.py:506: SimulationModeWarning: BlindAI is running in simulation mode. This mode is provided solely for testing purposes. It does not provide any security since there is no SGX enclave. The simulation mode MUST NOT be used in production.
  warnings.warn(


> For the purposes of this demo, we are running the server on `localhost` using the default ports, but you can modify the host and ports in the `connect()` function.

Finally, we can upload the model using the client `upload_model()` method! 

We'll need to specify the ONNX model's file name via the `model` parameter.

Then we can store and print out our `model_id`, which is used to identify the model when running or deleting the model.

In [ ]:
# AI company uploads model to server
response = client_1.upload_model(model="./resnet18.onnx")
PYTORCH_MODEL_ID = response.model_id
print(PYTORCH_MODEL_ID)

b20a226b-7f45-4361-98a4-71486362a53e


The PyTorch model has now been successfully uploaded to the BlindAI server and is ready to be consumed by users.

## TensorFlow model
__________________________

Let's now take a look at how we can convert `TensorFlow saved models` to ONNX using the [`tf2onnx.convert` tool](https://github.com/onnx/tensorflow-onnx). This is the recommended and most popular way to convert TensorFlow models to ONNX, but you can also convert to ONNX from different formats like `graphdef` or `checkpoint` format.

>For more information about converting TensorFlow models from `checkpoints` or `graphdef` format, please see the [`tf2onnx` documentation](https://github.com/onnx/tensorflow-onnx).

### Loading the model

We'll start by loading the built-in ResNet50 model, which is a variation on the ResNet18 model which is 50 layers deep, rather than 18.

By importing and initializing the ResNet50 class from the `keras.applications.resnet50` module, we get back a Keras model instance of the ResNet50 model. We then transform this into the SavedModel format by using the `save()` method and providing a name for our `SavedModel` directory.

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

# initalize pre-trained ResNet50 model
model = ResNet50(weights='imagenet')

# convert model to SavedModel format
model.save("my_model")

A `SavedModel` is a directory, containing a `saved_model.pb` file where the TensorFlow model is stored, as well as any additional required files. Here we create our `SavedModel` directory in our current working directory and call it "my model".



### ONNX Conversion

Now that we our model in `SavedModel` format we can use the `tf2onnx.convert` tool to convert our model to ONNX. We provide the path to our `saved-model` and the path we want our `output` onnx file to have.

In [ ]:
# convert SavedModel to onnx
!python -m tf2onnx.convert --saved-model ./my_model --output ./resnet_tf.onnx

2023-03-21 17:19:39.613849: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 17:19:39.614001: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 17:19:39.614050: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package

We now have a `resnet_tf.onnx` file in our current working directory ready to be uploaded to BlindAI.

### Upload the model

We are now ready to upload the model to the BlindAI server using the `upload_model` method.

We then print out the model's ID which would be used later to identify the model when performing operations in the BlindAI API.

In [ ]:
# AI company uploads model to server
client_1 = blindai_preview.core.connect(addr="localhost", simulation_mode=True, hazmat_http_on_unattested_port=True)
response = client_1.upload_model(model="./resnet_tf.onnx")
TF_MODEL_ID = response.model_id
print(TF_MODEL_ID)

/usr/local/lib/python3.9/dist-packages/blindai_preview/client.py:506: SimulationModeWarning: BlindAI is running in simulation mode. This mode is provided solely for testing purposes. It does not provide any security since there is no SGX enclave. The simulation mode MUST NOT be used in production.
  warnings.warn(


41608476-bb41-4dda-a7d2-b915db35f5bc


## HuggingFace model
_______________________________


`HuggingFace` provides a space where the AI community can share and collaborate on open-source models. 

In this section, we will show you how you can export models from the `HuggingFace hub`, directly in ONNX format. 

### Download and convert the model

If you are using a `PyTorch` or `TensorFlow` model downloaded from the `HuggingFace hub`, you could also use the conversion methods described in the [`PyTorch`](#pytorch-model) or [`TensorFlow`](#tensorflow-model) sections respectively.

But we will focus here on the method recommended in the `HuggingFace` documentation, which uses the `optimum` CLI to convert `"Transformer"` and `"Diffuser"` models to `ONNX` format.

>Note that not all architectures can be converted to ONNX format. For a full list of architectures compatible with ONNX conversion, see [the HuggingFace documentation](https://huggingface.co/docs/transformers/serialization#onnx).

We'll use `optimum-cli` to download and convert the `resnet50` model from the `HuggingFace hub` to `ONNX`. We'll do so by specifying the `model id` we wish to download and a path where we want all generated files to be stored.

In [ ]:
# Download model from hub in ONNX format
!optimum-cli export onnx --model distilbert-base-cased-distilled-squad huggingface_onnx/

2023-03-21 17:20:24.957761: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 17:20:24.957889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-21 17:20:24.957913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-21 17:20:34.601177: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could 

This will generate various files to the specified location including a `model.onnx` file which contains the downloaded model in `ONNX` format. This method also works with both `PyTorch` and `TensorFlow` models.

>Note that, when downloading models from the hub, the optimum-cli `onnx` export tool will perform automatic task detect. We can see this in the output of the command. If you want to turn automatic detection off or for local models, you can use the `--task` option to turn off automatic task detection and specify a task.

>*For additional information, see the [HuggingFace documentation](https://huggingface.co/docs/optimum/main/en/exporters/onnx/usage_guides/export_a_model#selecting-a-task).*

### Upload the model

Let's upload this model to the BlindAI server! We use the `upload_model()` method.

We then print out the model's ID which will be used later to identify the model when performing operations in the BlindAI API.

In [ ]:
# AI company uploads model to server
client_1 = blindai_preview.core.connect(addr="localhost", simulation_mode=True, hazmat_http_on_unattested_port=True)
ret = client_1.upload_model(model="./huggingface_onnx/model.onnx")
HF_MODEL_ID = ret.model_id
print(HF_MODEL_ID)

82fc6f57-5b28-49db-a677-85ab9e3e238e


The model has now been successfully uploaded to the BlindAI server and is ready to be consumed by users.

## Conclusions
____________________________________

This is the end of our tutorial on uploading models to BlindAI.

We have seen how to:

* **Convert** TensorFlow, PyTorch and HuggingFace model to ONNX format.
* **Upload** these models to BlindAI.

Please check out the rest of our [BlindAI documentation](https://blindai-preview.mithrilsecurity.io/en/latest/) to see more examples of how you can use BlindAI to deploy AI models without compromising the safety of user data or models.